In [2]:
#code sorted embedding pipeline
#unify datasources to one input dataframe: internatArchive, JSTORE, SCRIPT.ORG

import pandas as pd
import json
#anwenden des modells auf das erste fullText element
from transformers import BertModel, BertTokenizer, AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
import torch
import torch.nn.functional as F
import asyncio
from torch.utils.data import DataLoader, Dataset


In [3]:
# Load all three data sources

# Pfad: Path: /Users/merlin/Desktop/BA/Bachelorabeit Uni Leipzig/data/1880_nopsy.jsonl
# Pfad JSTOR Download mit Textdaten
path_social_science = '/Users/merlin/Desktop/BA/Bachelorabeit Uni Leipzig/programming/dhox/data/1880_nopsy.jsonl'
path_to_scraped = '/Users/merlin/Desktop/BA/Bachelorabeit Uni Leipzig/programming/final_scraped_data.json'
path_to_iarch = '/Users/merlin/Desktop/BA/Bachelorabeit Uni Leipzig/programming/iArchive/iarchive_data.json'

# Reader JSONL: In JSONL enthält jede Zeile eine JSON-Datei
# Pfad: Path: /Users/merlin/BA/mata_and_text.jsonl
#data_fullText = pd.read_json(path_social_science, lines=True)
df_scraped = pd.read_json(path_to_scraped, lines=True)
df_iarch = pd.read_json(path_to_iarch, lines=True)
df_iarch['year'] = df_iarch['extracted_year']
df_scraped#.columns, df_iarch.columns

,href,title,authors,journal,issue,date,doi,abstract,keywords,fullText,doi_count,year
0,/journal/paperinformation?paperid=125531,Research on the Construction of China’s Strate...,"[Lina Jing, Nana Jing]",Open Journal of Political Science,Vol.13 No.3,2023-06-12,10.4236/ojps.2023.133017,In response to the frequent\noccurrence of glo...,"[Global Security Governance, Strategic Communi...","1. Introduction At various levels, changes in ...",1,2023
1,/journal/paperinformation?paperid=125530,The Relationship between Inter-Organizational ...,"[Deng Gach Pal, Elijah Onyango Standslause Odh...",Open Journal of Political Science,Vol.13 No.3,2023-06-12,10.4236/ojps.2023.133016,The objective of this study was to examine the...,"[Leadership Performance, Political Interface, ...",1. Introduction As far as partnership is conce...,1,2023
2,/journal/paperinformation?paperid=125526,Deconstructivism as a Pedagogical Intervention...,"[Bridget Peak, Ayman Elsemary, Badar Almamari]",Voice of the Publisher,Vol.9 No.2,2023-06-12,10.4236/vp.2023.92004,This study analyzes deconstructivism as a peda...,"[Art, Deconstructivism, Culture, Oman]",1. Introduction: Oman Vision 2040 and the Impo...,1,2023
3,/journal/paperinformation?paperid=125525,Research on China Academic Social Sciences and...,[Lei Yi],Voice of the Publisher,Vol.9 No.2,2023-06-12,10.4236/vp.2023.92003,One of the most spectacular aspects of the cha...,"[CASHL, HSS, ULA]",1. Introduction One of the most spectacular as...,1,2023
4,/journal/paperinformation?paperid=125523,Does It All Harm the Same?—An Empirical Explor...,"[Klara Schuster, Tim Lindermayer, Lisann van P...",Psychology,Vol.14 No.6,2023-06-09,10.4236/psych.2023.146049,"With ever-increasing smartphone use, phubbing ...","[Phubbing, Smartphone Use, Ostracism, Exclusio...",1. Introduction Smartphones have become an ind...,1,2023
...,...,...,...,...,...,...,...,...,...,...,...,...
19121,/journal/paperinformation?paperid=52835,The Role of Self-Control on Mood States and He...,"[Pierluigi Diotaiuti, Filippo Petruccelli, Lui...",Psychology,Vol.5 No.19,2014-12-31,10.4236/psych.2014.519223,Blind\npeople face daily a stressful condition...,"[Self-Control, Self-Regulation, Mood States, H...",1. Introduction The loss of vision requires a...,1,2014
19122,/journal/paperinformation?paperid=52828,Could an Interactive-Balanced Environment for ...,"[Baha Makhoul, Raphiq Ibrahim]",Psychology,Vol.5 No.19,2014-12-31,10.4236/psych.2014.519222,A large\nnumber of the scientific efforts that...,"[Arabic, Reading Acquisition, Kindergarten, Li...","1. Introduction For most people, literacy dev...",1,2014
19123,/journal/paperinformation?paperid=52810,Developing Entrepreneurial Curriculum in Indon...,"[Achmad Room Fitrianto, Abd A’la]",Creative Education,Vol.5 No.24,2014-12-30,10.4236/ce.2014.524232,Entrepreneurship is believed as a method that ...,"[Entrepreneurial Education, Strategic Planning...",1. Introduction Indonesia experienced the hig...,1,2014
19124,/journal/paperinformation?paperid=52809,A Proposal for the Development of Pre-Primary ...,[Ali Naser Al-Mogbel],Creative Education,Vol.5 No.24,2014-12-30,10.4236/ce.2014.524231,This study aims to identify the most important...,"[Curriculum, Development, Pre-Primary]",1. Introduction The acceleration in scientifi...,1,2014


In [ ]:
#Load JSTORE JSONL with loop

input_stream = []
with open(path_social_science, 'r' ,encoding='utf-8') as f:
    try:
        for i, line in enumerate(f,1):
            input_stream.append(json.loads(line))
            
    except json.JSONDecodeError:
        print('skipped bad line {i}')

df_jstor = pd.DataFrame(input_stream)
#data_fullText = pd.read_json(path_social_science, lines=True)

In [ ]:
df_iarch.rename(columns={'content':'fullText'}, inplace=True)
df_scraped.columns, df_iarch.columns
data_fullText = pd.concat([df_scraped[['fullText','year']], df_iarch[['fullText', 'year']]]) #add df_jstor[['fullText','year']]

In [ ]:
# load finetuned model from storage
tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_model")

# Load model
model = AutoModel.from_pretrained("./fine_tuned_model")

In [ ]:
#method for splitting input text into chunks of 512 tokens

def chunk_text(text, max_len=512):
    tokens = tokenizer.encode(text, add_special_tokens=False)
    chunks = [tokens[i:i+max_len-2] for i in range(0, len(tokens), max_len-2)]
    return [tokenizer.build_inputs_with_special_tokens(chunk) for chunk in chunks]


#method for splitting input text into chunks of 512 tokens

def chunk_text(text, max_len=512):
    tokens = tokenizer.encode(text, add_special_tokens=False)
    chunks = [tokens[i:i+max_len-2] for i in range(0, len(tokens), max_len-2)]
    return [tokenizer.build_inputs_with_special_tokens(chunk) for chunk in chunks]

#method for splitting input text into chunks of 512 tokens for case of using bi-transformer model

def chunk_text_bi(text, max_len=512):
    tokens = tokenizer.encode(text, add_special_tokens=False)
    alphanumeric_tokens = [
        token for token in tokens 
        if tokenizer.decode([token]).strip().isalnum()
    ]
    chunks = [alphanumeric_tokens[i:i+max_len-2] for i in range(0, len(alphanumeric_tokens), max_len-2)]
    return [tokenizer.decode(chunk) for chunk in chunks]

#batch tokenizer properties
def collate_fn(batch):
    return tokenizer(
        batch,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512,
    )

class TextChunkDataset(torch.utils.data.Dataset):
    def __init__(self, chunks):
        self.chunks = chunks

    def __len__(self):
        return len(self.chunks)

    def __getitem__(self, idx):
        return self.chunks[idx]
    
#Method embTokenDf_bi: Takes a list of text chunks, tokenizes them, and returns a DataFrame with token embeddings
def embTokenDf_bi_parallel(chunks):

    # use dataloader for batching
    
    loader = DataLoader(
        TextChunkDataset(chunks),
        batch_size=16,  # tune this
        shuffle=False,
        collate_fn=collate_fn,
        pin_memory=True,
    )
    dfs = []
    with torch.no_grad():
        for batch in loader:
            output = model(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
            )

            for i in range(batch["input_ids"].size(0)):
                mask = batch["attention_mask"][i].bool()
                emb = output.last_hidden_state[i][mask]
                tokens = tokenizer.convert_ids_to_tokens(
                    batch["input_ids"][i][mask]
                )

                # Build dataframe
                df = pd.DataFrame(emb.cpu().numpy())
                df.insert(0, "token", tokens)
                dfs.append(df)

    final_df = pd.concat(dfs, ignore_index=True)
    print(f"✔ Processed chunks: {len(tokens)} ")

    return final_df

#Method embTokenDf_bi: Takes a list of text chunks, tokenizes them, and returns a DataFrame with token embeddings
async def embTokenDf_bi_starter(chunks):
    
    tasks = [embTokenDf_bi_async_with_limit(chunk) for chunk in chunks]
    
    results = await asyncio.gather(
        *tasks,
        return_exceptions=True,
    )

    results = [] 
    for res in results:
        if isinstance(res, Exception):
            print(f"Error processing chunk: {res}")
        results.append(res)
    output = pd.concat(results, ignore_index=True) if results else pd.DataFrame()
        

    return output

# Function to average token embeddings
def average_token_embeddings(df):
    embedding_cols = df.columns[1:]  # assuming first col is 'token'
    df_avg = df.groupby("token")[embedding_cols].mean().reset_index()
    return df_avg

# compute dinstances between two DataFrames, based on a list of similar tokens
def compute_distances(df1, df2, similars):
    distances = []
    for token in similars:
        if token in df1['token'].values and token in df2['token'].values:
            emb1 = torch.tensor(df1[df1['token'] == token].iloc[:, 1:].values.flatten(), dtype=torch.float)
            emb2 = torch.tensor(df2[df2['token'] == token].iloc[:, 1:].values.flatten(), dtype=torch.float)
            cos_sim = F.cosine_similarity(emb1, emb2, dim=0)
            distances.append((token, cos_sim))
    return pd.DataFrame(distances, columns=['token', 'distance'])


#parrallelize embTokenDf_bi limit 5 concurrency 

async def embTokenDf_bi_async(text: str):
    return await asyncio.to_thread(embTokenDf_bi_parallel, text)

NLP_CONCURRENCY = 5
sem = asyncio.Semaphore(NLP_CONCURRENCY)

async def embTokenDf_bi_async_with_limit(chunk: str):
    async with sem:
        return await embTokenDf_bi_async(chunk)

def year_to_decade(year):
    return (year // 10) * 10

# example for one decade
data_fullText['decade'] = data_fullText['year'].apply(year_to_decade)
chunks_60s=[]
for _ in data_fullText[data_fullText['decade']==1960]['fullText']:
    chunks_60s.extend(chunk_text_bi(_))

In [ ]:
# compute embeeings with
await embTokenDf_bi_starter(chunks_60s)